In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from helpers import *

## Load the training data into feature matrix, class labels, and event ids:

In [67]:
from proj1_helpers import *
DATA_TRAIN_PATH = '' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data("../data/train.csv")

## Data Analyzing

In [68]:
# As we can see here, y only takes value -1 or 1:
for value in y:
    assert(value==1 or value==-1)
print("All value in y is equal either to 1 or -1.")


All value in y is equal either to 1 or -1.


This means that y is a binary variable. So should we modify y's coding to {0, 1} instead of {-1, 1} if we want the logistic regression methods to work?
Note that at first sight, logistic regression seems to be the best solution to fit the data since this method was designed for binary classification.

## Data Cleaning
We have to handle:
- outliers:
A value is considered as an outlier if it does not fit in a range defined from quartiles. Outliers are replaced by the mean value of the observations.
- unasssigned values (-999, 999):
We proceed the same way


In [69]:
for col in tX.T:
    q1 = np.percentile(col,25)
    q3 = np.percentile(col,75)
    interq = q3-q1
    
    col_cleaned = col[abs(col)!=999]
    col_cleaned = col_cleaned[col_cleaned<=q3+interq]
    col_cleaned = col_cleaned[col_cleaned>=q1-interq]
    mean = np.mean(col_cleaned)
    
    col[abs(col)==999] = mean
    col[col>q3+interq] = mean
    col[col<q1-interq] = mean
    
tX,_,_ = standardize(tX)

### Using PCA to get rid of features that don't give enough information

In [70]:
#C = 1/tX.shape[0]*tX.T.dot(tX)
#  
#eigenvalue, eigenvector = np.linalg.eig(C)
#SUM = np.sum(eigenvalue)
#
#idx = np.argsort(eigenvalue)[::-1]
#eigenvector = eigenvector[:,idx]
#eigenvalue = eigenvalue[idx]
#
#F = 0
#k = 0
#while F < 0.95:
#    F += eigenvalue[k]/SUM
#    k = k+1   
#
#eigenvector=eigenvector[:, :k]
#
#tX = np.dot(eigenvector.T, tX.T).T
#
#print(tX.shape)

## Some Helper Functions


In [71]:
# Some helper function:

def compute_gradient(y, tx, w):
    """Compute the gradient."""
    N = len(y)
    e = y - tx.dot(w)
    return (-1/N)*(tx.T).dot(e)

def compute_loss(y, tx, w):
    """Compute the cost by MSE"""
    N = len(y)
    e = y - tx.dot(w)
    return (1/(2*N))*((e.T).dot(e))

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    # concatenate column of ones of size (25000, 1)
    result = np.ones((x.shape[0], 1))
    for i in range(1, degree+1):
       result = np.concatenate((result, x ** i), axis=1)
    return result

def split_data(x, y, ratio, seed=1):
    """Randomly splits the data given in input into two subsets (test/train).
    The ratio determines the size of the training set."""
    np.random.seed(seed)
    size = y.shape[0]
    # randomly permutes array of intergers from 0 to size-1
    indexes = np.random.permutation(size)
    tr_size = int(np.floor(ratio * size))
    # get (randomly generated) indexes of training/testing set
    tr_indexes = indexes[0:tr_size]
    te_indexes = indexes[tr_size:]
    # split x (resp. y) into two subarrays x_tr, x_te (resp. y_tr, y_te)
    x_tr = x[tr_indexes]
    y_tr = y[tr_indexes]
    x_te = x[te_indexes]
    y_te = y[te_indexes]
    return [x_tr, y_tr, x_te, y_te]

def sigmoid_scal(t):
    """apply sigmoid function on t."""
    if(t > 0):
        return np.exp(t)/(1+np.exp(t))
    else:
        return 1/(1+np.exp(-t))
    
sigmoid = np.vectorize(sigmoid_scal)
    

def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    loss = 0
    for index, row in enumerate(tx):
        loss += np.log(1+np.exp(row.dot(w)))-y[index]*row.dot(w)
    return loss

def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T.dot(sigmoid(tx.dot(w))-y)

def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """   
    grad = calculate_gradient(y, tx, w)
    loss = calculate_loss(y, tx, w)
    #f = calculate_loss(y, tx, w-grad)
    #print(f)
                                
    w -= gamma*grad
    next_gamma = gamma/2 #next_step_size(f, loss, grad, gamma)
    
    return w, loss, next_gamma

def next_step_size(f, loss, grad, gamma, maxiter=50, beta=1/2):
    alpha = gamma
    i=0
    while (f > loss-alpha/2*np.square(np.linalg.norm(grad)) and i<maxiter):
        print(f)
        alpha *= beta
        i += 1
    return alpha           
                                                

def error(y,y_est):
    """Computes the percentage of right values in the 
    regression result compared to our test data"""
    diff = np.count_nonzero(y-y_est)/len(y)
    return (1-diff)*100
    

## Split the data into two parts

In [72]:
tX, y, tX_te, y_te = split_data(tX, y, 2/3)

In [73]:
print(tX.shape)

(166666, 31)


## Functions to implement

In [74]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """performs linear regression using gradient descent algorithm. 
    Returns two arrays containing weights and loss values 
    at each step of the algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and loss
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        #update w
        w = w - gamma * gradient
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
    return w, loss


def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """performs linear regression using stochastic gradient descent algorithm. 
    Returns two arrays containing weights and loss values 
    at each step of the algorithm."""
    ws = [initial_w]
    losses = []
    w = initial_w
    batch_size = 800 #try changing batch size
    
    for n_iter in range(max_iters):
        print("iteration", n_iter)
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gradient = compute_gradient(minibatch_y, minibatch_tx, w)
            #compute new loss and w
            loss = compute_loss(y, tx, w) # add one loss per minibatch (compute mean)
            w = w - gamma * gradient
            # store loss and w in arrays
            ws.append(w)
            losses.append(loss)
                 
    return w, loss

def least_squares(y, tx):
    """performs linear regression by calculating 
    the least squares solution using normal equations.
    returns loss and optimal wieghts."""
    opt_w = np.linalg.inv(tx.T.dot(tx)).dot(tx.T).dot(y)
    #computes the loss using MSE
    mse = compute_loss(y, tx, opt_w)
    return opt_w, mse
    
def ridge_regression(y, tx, lambda_):
    """implement ridge regression. This calculates the MSE while taking in 
    accout a regularizer that is determined by lambda. This has for effect to
    penalize/avoid large weights in order to avoid overfitting."""
    # tx is the polynomial basis
    opt_w = (np.linalg.inv(tx.T.dot(tx)+lambda_*2*len(y)*np.identity(tx.shape[1])).dot(tx.T)).dot(y)
    mse = compute_loss(y, tx, opt_w)
    return opt_w, mse

def logistic_regression(y, tx, max_iter, gamma=1e-5):
    threshold = 1e-10
    losses = np.array([0.,1.])
    w = np.zeros(tx.shape[1])
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        w, loss, gamma = learning_by_gradient_descent(y, tx, w, gamma)
        losses[1] = losses[0]
        losses[0] = loss
        if iter % 1 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        if abs(losses[0]-losses[1]) < abs(losses[1])*threshold:
            break
        
    print("The loss={l}".format(l=calculate_loss(y, tx, w)))
    return w, loss

    
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    #TODO
    raise NotImplementedError


In [75]:
### logistic regression test
max_iter = 1000
y01 = (y+1)/2


y_red = y01#[0:10000]
x_red = tX#[0:10000]

w,loss = logistic_regression(y_red, x_red, max_iter)
print(error(y_te,predict_labels(w,tX_te)))

Current iteration=0, the loss=115524.06799497179
Current iteration=1, the loss=89590.2970742534
Current iteration=2, the loss=85555.18430650537
Current iteration=3, the loss=84683.39592008777
Current iteration=4, the loss=84322.517953193
Current iteration=5, the loss=84155.7869616509
Current iteration=6, the loss=84075.45585587645
Current iteration=7, the loss=84036.00830793372
Current iteration=8, the loss=84016.4593791849
Current iteration=9, the loss=84006.72806751641
Current iteration=10, the loss=84001.8731314198
Current iteration=11, the loss=83999.44833483384
Current iteration=12, the loss=83998.23660335607
Current iteration=13, the loss=83997.63090418746
Current iteration=14, the loss=83997.32809622944
Current iteration=15, the loss=83997.17670265505
Current iteration=16, the loss=83997.10100847056
Current iteration=17, the loss=83997.06316202674
Current iteration=18, the loss=83997.04423896824
Current iteration=19, the loss=83997.03477747751
Current iteration=20, the loss=8399

## Use Polynomial Regression to find the optimal degree

In [ ]:
def polynomial_regression():
    """For each degree, constructs the polynomial basis function expansion of the data
       and stores the corresponding RMSE in an array. At the end we chose the degree that
       generated the smallest RMSE. Of course we cannot test all degrees so this is not
       optimal but it helps us having a good idea of the optimal degree value."""
    # define parameters
    degrees = [1, 2, 3, 4, 5]
    
    # for each degree we store the corresponding RMSE in this array
    rmse_array = np.array([])

    for ind, degree in enumerate(degrees):
        # form the data to do polynomial regression:
        polynomial_basis = build_poly(tX, degree)
        
        # least square and calculate rmse:
        mse, weight = least_squares(y, polynomial_basis)
        rmse = np.sqrt(2*mse)
        rmse_array = np.append(rmse_array, rmse)
        print("RMSE for degree", degree, ":", rmse)
    
    # plot the RMSE in function of the degree
    plt.plot(degrees, rmse_array)
    plt.xlabel('degree')
    plt.ylabel('RMSE')
    
    #compute the best degree
    best_degree = degrees[np.argmin(rmse_array)]
    print("The best degree among those we tested is", best_degree, ".")

polynomial_regression()

### Results
Looking at the results, it seems like 2 is the optimal degree. However, we might be overfitting the data because there is no regularization step in polynomial_regression. Thus we'll use the Ridge regression, which uses a regularizer that depends on a parameter lambda.
We'll compute the RMSE for different lambda and degree values in order to determine the best ones.

## Use Ridge Regression to determine optimal lambda

In [ ]:
def ridge_regression_demo(x, y, ratio, seed):
    """Calculate polyomial basis tX from x with given degree,
    splits the data according to given ratio and then run
    ridge regression on tX, y with different lambda values.
    At the end we plot the RMSEs of training/testing set in
    function of lambda in order to determine the best lambda value"""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    degrees = [1, 2]
    
    # split the data, and return train and test data:
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    #calculate test/train RMSE for each lambda and store them in lists
    rmse_list_tr = np.empty([len(degrees), len(lambdas)])
    rmse_list_te = np.empty([len(degrees), len(lambdas)])
    optimal_weights = np.empty([np.shape(tX)[1]])
    best_RMSE = 10e10
    for i, degree in enumerate(degrees):
        # for each lambda, store the best RMSE and the degree that generated it
        for j, lambd in enumerate(lambdas):
            # compute polynomial basis from given degree
            poly_basis_tr = build_poly(x_tr, degree)
            poly_basis_te = build_poly(x_te, degree)
            # compute training and testing (R)MSE for current lambda/degree
            w_tr, mse_tr = ridge_regression(y_tr, poly_basis_tr,lambd)
            mse_te = compute_loss(y_te, poly_basis_te, w_tr)
            rmse_tr = np.sqrt(2*mse_tr)
            rmse_te = np.sqrt(2*mse_te)
            #print("Training RMSE for lambda =", lambd, "and degree", degree, ":", rmse_tr, "\n")
            #print("Testing RMSE for lambda =", lambd, "and degree", degree, ":", rmse_te, "\n")
            # Store RMSEs in arrays
            rmse_list_tr[i][j] = rmse_tr
            rmse_list_te[i][j] = rmse_te
            # we do this to get optimal weights
            if(best_RMSE > rmse_tr):
                best_RMSE = rmse_tr
                optimal_weights = w_tr
    
    # get best degree, lambda, RMSE
    degree_index_tr, lambd_index_tr = np.where(rmse_list_tr == rmse_list_tr.min())
    degree_index_tr, lambd_index_tr = (degree_index_tr[0],lambd_index_tr[0])
    best_rmse_tr = rmse_list_tr[degree_index_tr][lambd_index_tr]
    print("Best training RMSE is", best_rmse_tr, "with degree", degrees[degree_index_tr], "and lambda=", lambdas[lambd_index_tr], "\n")
    return optimal_weights, best_RMSE
    
seed = 1
split_ratio = 0.8
w, loss = ridge_regression_demo(tX, y, split_ratio, seed)

## Computing the weights with different methods

In [60]:
#initial values
gamma = 1e-7
initial_w = 0*np.ones(len(tX[0])) #try changing initial w
max_iters = 10
poly_basis = build_poly(tX, 2)

### Least Squares

In [61]:
w, loss = least_squares(y, tX)
print(error(y_te,predict_labels(w,tX_te)))

77.17858257133943


### Least Squares with polynomial basis

In [62]:
w, loss = least_squares(y, poly_basis)
print(error(y_te,predict_labels(w,tX_te)))

ValueError: shapes (83334,25) and (51,) not aligned: 25 (dim 1) != 51 (dim 0)

### Least Squares - Gradient Descent

In [63]:
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(error(y_te,predict_labels(w,tX_te)))

72.57781937744498


### Least Squares - Stochastic Gradient Descent

In [64]:
w, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma)
print(error(y_te,predict_labels(w,tX_te)))

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
72.57781937744498


### Ridge Regression

In [ ]:
# to find optimal degree and lambda, check "Use RR to determine optimal
# lambda and degree" section
degree = 2
poly_basis = build_poly(tX, 2)
lambda_ = 1e-05
w, loss = ridge_regression(y, poly_basis, lambda_)
print(error(y_te,predict_labels(w,tX_te)))

### Ridge regression with data splitting and chosing best lambda/deg

In [ ]:
seed = 1
split_ratio = 0.8
w, loss = ridge_regression_demo(tX, y, split_ratio, seed)
print(error(y_te,predict_labels(w,tX_te)))

In [ ]:
print(tX_test.shape)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
print("done")

In [ ]:
base_poly_x = build_poly(tX_test, 2)
poly_basis = build_poly(tX, 2)
w, loss = least_squares(y, poly_basis)
print("done")

In [ ]:
OUTPUT_PATH = '../data/submissionData/logisticRegression1.csv' # TODO: fill in desired name of output file for submission

o = np.ones((tX_test.shape[0],1))
tX_test,_,_ = standardize(tX_test)

y_pred = predict_labels(w, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)
print("done")

In [ ]:
print(y_pred.shape)